In [1]:
import os
import sys
srcpath = os.path.realpath('../SourceFiles')
sys.path.append(srcpath)
from teproteus import TEProteusAdmin as TepAdmin
from teproteus import TEProteusInst as TepInst

In [2]:
%matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
from IPython.core.debugger import set_trace
from scipy.signal import chirp, sweep_poly
import matplotlib.pyplot as plt

In [3]:
pid = os.getpid()
print('process id {0}'.format(pid))

# Connect to instrument

sid = 4
Debug = False

# Use lib_dir_path = None 
# for default location (C:\Windows\System32)
# Change it only if you know what you are doing

lib_dir_path = None
#lib_dir_path = r'C:\Tabor\Proteus\ProteuaAwg-10349-Costumer'


admin = TepAdmin(lib_dir_path)
inst = admin.open_instrument(slot_id=sid)

inst.default_paranoia_level = 2 # 0, 1 or 2

process id 6820


In [4]:

#get hw option
inst.send_scpi_query("*OPT?")
# reset - must!
inst.send_scpi_cmd( "*RST")
    
    
# Get the DAC mode (8 bits or 16 bits)
resp = inst.send_scpi_query(':SYST:INF:DAC?')
dac_mode = resp
print('DAC mode {0} '.format(dac_mode))

if dac_mode == 'M0':
    max_dac = 65535
    data_type = np.uint16 
else:
    max_dac = 255
    data_type = np.uint8 
    
half_dac = max_dac // 2.0

# Get number of channels
resp = inst.send_scpi_query(":INST:CHAN? MAX")
print("Number of channels: " + resp)
num_channels = int(resp)

# Get the maximal number of segments
resp = inst.send_scpi_query(":TRACe:SELect:SEGMent? MAX")
print("Max segment number: " + resp)
max_seg_number = int(resp)

# Get the available memory in bytes of wavform-data (per DDR):
resp = inst.send_scpi_query(":TRACe:FREE?")
arbmem_capacity = int(resp)
print("Available memory per DDR: {0:,} wave-bytes".format(arbmem_capacity))


    
    



DAC mode M0 
Number of channels: 4
Max segment number: 65536
Available memory per DDR: 4,286,578,688 wave-bytes


In [5]:
seglen=102400000
num_cycles=200000
cyclelen = seglen / num_cycles

x = np.linspace(start=0, stop=2 * np.pi * num_cycles, num=seglen, endpoint=False)
y = (np.sin(x) + 1.0) * half_dac
y = np.round(y)
y = np.clip(y, 0, max_dac)
if dac_mode == 'M0':
    wave = y.astype(np.uint16)
else:
    wave = y.astype(np.uint8)
        
# data-type code: 'B' = uint8, 'H' = uint16
data_type = 'H' if dac_mode == 'M0' else 'B'


channb=1
segnum=1

cmd = ':INST:CHAN {0}'.format(channb)
inst.send_scpi_cmd(cmd)

cmd = ':TRAC:DEF {0}, {1}'.format(segnum, seglen)
inst.send_scpi_cmd(cmd)

cmd = ':TRAC:SEL {0}'.format(segnum)
inst.send_scpi_cmd(cmd)

inst.timeout = 30000

tic = time.perf_counter()

inst.write_binary_data(':TRAC:DATA', wave)

#print('Downloaded the wave in "{0}" seconds'.format(time_lapsed))

toc = time.perf_counter()
timer = float(toc-tic)

print('Downloaded the wave in "{0}" seconds'.format(timer))


resp = inst.send_scpi_query(':SYST:ERR?')
resp = resp.rstrip()
if not resp.startswith('0'):
     print('ERROR: "{0}" after writing binary values'.format(resp))

cmd = ':SOUR:FUNC:MODE:SEGM {0}'.format(segnum)
inst.send_scpi_cmd(cmd)

Downloaded the wave in "0.2554786" seconds


0

In [6]:
# close connection
inst.close_instrument()
admin.close_inst_admin()

0